In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
tqdm.pandas()

In [6]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 transformers==4.32.1 datasets==2.14.4 peft==0.5.0 sentencepiece accelerate bitsandbytes --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


In [8]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 

In [7]:
from sentence_transformers import SentenceTransformer, util
from inference import LLM

ModuleNotFoundError: ignored

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
trained_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

NameError: ignored

In [9]:
movies = pd.read_csv("Shared with me/CS 583 Project/datasets/movie_datasets/imdb/movie_complete_clean.csv")

FileNotFoundError: ignored

In [ ]:
movies.head()

,tconst,originalTitle,startYear,genres,runtimeMinutes,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,Romance,45,5.3,207.0
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,"Documentary,News,Sport",100,5.3,484.0
2,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",70,6.0,855.0
3,tt0000591,L'enfant prodigue,1907,Drama,90,5.0,21.0
4,tt0000615,Robbery Under Arms,1907,Drama,\N,4.3,25.0


In [ ]:
def generate_embeddings(string):
    return trained_model.encode(string, convert_to_tensor=True, normalize_embeddings=True, show_progress_bar=True)

In [ ]:
search_text = lambda details: f'The movie {details["originalTitle"]} was released in the year {details["startYear"]} is a {details["genres"]} with a  runtime of {details["runtimeMinutes"]} minutes'
movie_text = lambda details: f'{details["originalTitle"]}  ({details["startYear"]})'

In [ ]:
movies["search_text"] = movies.apply(search_text, axis=1)
movies["movie_text"] = movies.apply(movie_text, axis=1)

In [ ]:
# movie_embeddings = generate_embeddings(movies["movie_text"].to_list())

(499864, 384)

In [ ]:
# np.save("./datasets/movie_datasets/imdb/movie_name_embeddings.npy", movie_embeddings.detach().cpu().numpy())

In [ ]:
movie_embeddings = np.load("./datasets/movie_datasets/imdb/movie_name_embeddings.npy", allow_pickle=True)
movie_embeddings = torch.from_numpy(movie_embeddings).to(device)

What movie would you recommend after watch
I like <movie>, could you recommend me 3 similar ones?


In [ ]:
def retreive_top_k_results(question, topk=1):
    question_embeddings = generate_embeddings(question)
    result =  util.semantic_search(question_embeddings, movie_embeddings, top_k=topk)[0][0]
    return  result["score"], result["corpus_id"]

In [ ]:
question = "Who were the stars in The Avengers 2012 the power?"

In [ ]:
score, id = retreive_top_k_results(question)
print(movies["originalTitle"].iloc[id], movies["tconst"].iloc[id], score)
tconst = movies["tconst"].iloc[id]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Avengers tt0848228 0.7060876488685608


In [ ]:
from crawler import crawl_url
from utils import extract_additional_information, clean_synopsis

In [ ]:
movies_info = movies.set_index("tconst").copy()
movies_info.columns

Index(['originalTitle', 'startYear', 'genres', 'runtimeMinutes',
       'averageRating', 'numVotes', 'search_text', 'movie_text'],
      dtype='object')

In [ ]:
def generate_template(tconst, question):
    # short_desc_url = f"https://www.imdb.com/title/{tconst}"
    # main_html = crawl_url(short_desc_url)
    long_desc_url = (
        f"https://www.imdb.com/title/{tconst}/plotsummary/?ref_=tt_ov_pl"
    )
    plot_html = crawl_url(long_desc_url)
    synopsis = extract_additional_information(plot_html)
    synopsis = clean_synopsis(synopsis)
    details = movies_info.loc[tconst]
    return f"""
Below is a question regarding movies and shows paired with an input that provides further context. Write a response that appropriately completes the request.
###Instruction: {question}
###Input: Description: {synopsis} \n Release Year: {details["startYear"]} \n Genre: {details["genres"]} \n Runtime: {details["runtimeMinutes"]} minutes \n Rating: {details["averageRating"]}
###Response:
"""




In [ ]:
prompt = generate_template(tconst, question)
print(prompt)


Below is a question regarding movies and shows paired with an input that provides further context. Write a response that appropriately completes the request.
###Instruction: Who were the stars in The Avengers 2012 the power?
###Input: Description: Nick Fury (Samuel L. Jackson), director of S.H.I.E.L.D. (Strategic Homeland Intervention, Enforcement, and Logistics Division), arrives at S.H.I.E.L.D. headquarters outside of Santa Fe, New Mexico, during an evacuation. The Tesseract, an energy source of unknown potential, has activated. It opens a portal through space and the exiled Norse god Loki (Tom Hiddleston) steps through, carrying a strange spear with a blue glowing tip. Loki takes the Tesseract and uses the spear to take control of the minds of several SHIELD personnel, including Dr. Erik Selvig (Stellan Skarsgård) , and Agent Clint "Hawkeye" Barton (Jeremy Renner), to aid him in his getaway. SHIELD personnel pull out of their base when an energy surge from the Tesseract causes the 

In [ ]:
llm = LLM("openlm-research/open_llama_3b_v2", "./openlm-research-open_llama_3b_v2/")

c:\Users\smend\miniconda3\envs\stevens\Lib\site-packages\transformers\utils\hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
print(llm.generate(prompt))

Token indices sequence length is longer than the specified maximum sequence length for this model (3519 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


: 